In [ ]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, '../rtsvg/') # base location of the .py classes
from rtsvg import *
rt = RACETrack()

from transformers import pipeline

In [ ]:
#
# Predict with the base model.
#
model,tokenizer,device = rt.textCreateRoBERTaModel()
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=device
)
fill_mask("Apollo <mask> launched from Kennedy Space Center.")

In [ ]:
_text = """Apollo 13 (April 11–17, 1970) was the seventh crewed mission in the Apollo space program and the third meant to land on the Moon. The craft was launched from Kennedy Space Center on April 11, 1970, but the lunar landing was aborted after an oxygen tank in the service module (SM) failed two days into the mission. The crew instead looped around the Moon in a circumlunar trajectory and returned safely to Earth on April 17. The mission was commanded by Jim Lovell, with Jack Swigert as command module (CM) pilot and Fred Haise as Lunar Module (LM) pilot. Swigert was a late replacement for Ken Mattingly, who was grounded after exposure to rubella.
A routine stir of an oxygen tank ignited damaged wire insulation inside it, causing an explosion that vented the contents of both of the SM's oxygen tanks to space. Without oxygen, needed for breathing and for generating electric power, the SM's propulsion and life support systems could not operate. The CM's systems had to be shut down to conserve its remaining resources for reentry, forcing the crew to transfer to the LM as a lifeboat. With the lunar landing canceled, mission controllers worked to bring the crew home alive.
Although the LM was designed to support two men on the lunar surface for two days, Mission Control in Houston improvised new procedures so it could support three men for four days. The crew experienced great hardship, caused by limited power, a chilly and wet cabin and a shortage of potable water. There was a critical need to adapt the CM's cartridges for the carbon dioxide scrubber system to work in the LM; the crew and mission controllers were successful in improvising a solution. The astronauts' peril briefly renewed public interest in the Apollo program; tens of millions watched the splashdown in the South Pacific Ocean on television.
An investigative review board found fault with preflight testing of the oxygen tank and Teflon being placed inside it. The board recommended changes, including minimizing the use of potentially combustible items inside the tank; this was done for Apollo 14. The story of Apollo 13 has been dramatized several times, most notably in the 1995 film Apollo 13 based on Lost Moon, the 1994 memoir co-authored by Lovell – and an episode of the 1998 miniseries From the Earth to the Moon."""
_summary_bard    = """Apollo 13 was a crewed mission to the Moon that was aborted after an oxygen tank failure.  The crew was forced to transfer to the Lunar Module as a lifeboat and improvise new procedures to survive.  The astronauts faced great hardship, including limited power, a chilly and wet cabin, and a shortage of potable water.  The crew successfully returned to Earth and the story was dramatized in the film Apollo 13."""
_summary_chatgpt = """Apollo 13 was the seventh crewed mission in the Apollo space program and was intended to land on the Moon. However, a failed oxygen tank in the service module forced the crew to abandon the lunar landing. They instead orbited around the Moon and returned safely to Earth on April 17, 1970. The mission faced numerous challenges, including limited resources, a damaged spacecraft, and the need to adapt the lunar module to support three astronauts instead of two. The story of Apollo 13 gained widespread attention and has been portrayed in various dramatizations, including the 1995 film Apollo 13."""
model, tokenizer, device = rt.__textTrainRoBERTaModel__(_text)

In [ ]:
#
# Predict with the fine-tuned model
#
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=device
)
fill_mask("Apollo <mask> launched from Kennedy Space Center.")

In [ ]:
text_main      = _text
text_summaries = {'Bard':_summary_bard,'ChatGPT':_summary_chatgpt}
# text_summaries = {'ChatGPT':_summary_chatgpt}
methodology    = 'roberta_top_n'
main_txt_h     = 14
summary_txt_h  = 14
spacing        = 10
opacity        = 0.6
w              = 1024

summary_w = (w - spacing)/2

if model is None:
    model, tokenizer, device = rt.__textTrainRoBERTaModel__(text_main)

def ithColor(ith):
    if ith is None:
        _co_ = '#ff0000'
    elif ith == 0:
        _co_ = '#000000'
    elif ith < 3:
        _co_ = '#a0a0a0'
    elif ith < 5:
        _co_ = '#2ca25f'
    elif ith < 10:
        _co_ = '#ffeda0'
    else:
        _co_ = '#ffa500'
    return _co_

def summaryHighlights(_txt_):
    _lu_ , i0 = {}, 0
    for _part_ in _txt_.split('\n'):
        _stats_ = rt.__textRoBERTaStats__(_part_, model, tokenizer, device)
        for _stat_ in _stats_:
            ith = _stat_['ith']
            if ith is None or ith > 0:
                _lu_[(i0+_stat_['i0'],i0+_stat_['i1'])] = ithColor(ith)
        i0 += len(_part_) + 1
    return _lu_

_svgs_ = []
for _summary_desc_ in text_summaries:
    _summary_ = text_summaries[_summary_desc_]
    _rttb_ = rt.textBlock(_summary_, word_wrap=True, txt_h=summary_txt_h, w=summary_w)
    _svgs_.append(f'<svg x="0" y="0" width="{summary_w}" height="{24}">' + \
                  f'<rect x="0" y="0" width="{summary_w}" height="{24}" fill="#000000" />' + \
                  rt.svgText(_summary_desc_, 3, 20, txt_h=(summary_txt_h+4), color='#ffffff') + '</svg>')
    _highlights_ = summaryHighlights(_summary_)
    _build_ = _rttb_.wrap(_rttb_.background() + 
                          _rttb_.highlights(_highlights_,opacity=opacity) + 
                          _rttb_.unwrappedText())
    _svgs_.append(_build_)
    _svgs_.append(f'<svg x="0" y="0" width="{spacing}" height="{spacing}"></svg>')

rt.displaySVG(rt.tile(_svgs_,horz=False))

In [ ]:
def smallDistros(_sentence_):
    _stats_ = rt.__textRoBERTaStats__(_sentence_, model, tokenizer, device, bins=40)
    _bins_  = len(_stats_[0]['counts'])
    _h_     = 12
    _w_     = (_bins_ + 8) * len(_stats_)
    svg     = f'<svg x="0" y="0" width="{_w_}" height="{_h_ + 18}">'
    svg    += f'<rect x="0" y="0" width="{_w_}" height="{_h_ + 18}" fill="#ffffff" />'
    x       = 0
    for _stat_ in _stats_:
        _counts_ = _stat_['counts']
        _bounds_ = _stat_['boundaries']
        _score_  = _stat_['score']
        _ith_    = _stat_['ith']
        _max_    = max(_counts_)
        _here_   = None
        _co_     = ithColor(_ith_)
        for i in range(_bins_):
            h = (_h_ - 4) * _counts_[i] / _max_
            h = max(h,1)
            svg += f'<rect x="{4+x+i}" y="{_h_ - 2 - h}" width="1" height="{h}" fill="{_co_}" />'
            if _score_ >= _bounds_[i] and _score_ <= _bounds_[i+1]:
                _here_ = 4+x+i
        svg += f'<line x1="{_here_}" y1="{0}" x2="{_here_}" y2="{_h_-4}" stroke="#000000" />'
        svg += rt.svgText(str(_ith_), x=x+4+_bins_/2, y=_h_+6, color=_co_, anchor='middle', txt_h=10)
        svg += rt.svgText(_stat_['token'], x=x+4+_bins_/2, y=_h_+16, anchor='middle', txt_h=10)
        x += _bins_ + 8
    svg +=  '</svg>'
    return svg

_sentences_ = [
    'Apollo 13 launched from Kennedy Space Center to land on the Moon.',
    'Apollo 8 launched from Kennedy Space Center to land on the Moon.',
    'Mercury 13 launched from Kennedy Space Center to land on the Moon.',
    'Apollo 13 escaped from Kennedy Space Center to land on the Moon.',
    'Apollo 13 elephant from Kennedy Space Center to land on the Moon.',
    'Apollo 13 swam from the UK to arrive in France.',
    'Apollo 13 launched from Kennedy Space Center to land on the Sun.',
    'Apollo 13 launched from Kennedy Space Center to orbit another galaxy.',
    'Apollo 13 ran from Kennedy Space Center to land on the Moon.',
    'Apollo 13 flew from Kennedy Space Center to land on the Moon.',
    'Apollo 13 icy from Kennedy Space Center to land on the Moon.'
]
_svgs_ = []
for _sentence_ in _sentences_:
    _svgs_.append(smallDistros(_sentence_))
rt.displaySVG(rt.tile(_svgs_, horz=False))

In [ ]:
s  = 'This is a test.  This is\na split line\n\n\nwith more splits.  And now a new sentence.\nNext!'
r  = rt.textExtractSentences(s)
lu = {}
for x in r:
    lu[(x[1],x[2])] = x[0]
rt.displaySVG(rt.textBlock(s, word_wrap=True, line_space_px=1).underlines(lu))